In [2]:
from IPython.core.debugger import set_trace
import copy
from timeit import default_timer as timer

In [313]:
def get_input(test_data=None, dim=2):
    data = test_data if test_data is not None else open('input.txt').read()
    state = {}
    lists = [list(n) for n in data.split('\n')]
    for i in range(len(lists)):
        for j in range(len(lists[0])):
            tup = (i, j, 0) if dim==2 else (i,j,0,0)
            state[tup] = (lists[i][j] == '#')
    return state

In [235]:
def get_input_2(test_data=None):
    data = test_data if test_data is not None else open('input.txt').read()
    state = {}
    lists = [list(n) for n in data.split('\n')]
    for i in range(len(lists)):
        for j in range(len(lists[0])):
            state[(i, j, 0, 0)] = (lists[i][j] == '#')
    return state

In [187]:
test_data = '''.#.
..#
###'''

In [188]:
state = get_input()

In [189]:
get_input(test_data)

{(0, 0, 0): False,
 (0, 1, 0): True,
 (0, 2, 0): False,
 (1, 0, 0): False,
 (1, 1, 0): False,
 (1, 2, 0): True,
 (2, 0, 0): True,
 (2, 1, 0): True,
 (2, 2, 0): True}

In [309]:
def make_closure(n):
    if len(n) == 3:
        return lambda i,j,k: (i+n[0], j+n[1], k+n[2])
    elif len(n) == 4:
        return lambda i,j,k,l: (i+n[0], j+n[1], k+n[2], l+n[3])
    else:
        raise NotImplementedError
        
def prepare_dict(dim):
    nei_dict = {}
    for s,n in enumerate(itertools.product([1, 0, -1], repeat=dim)):
        if n!=tuple([0]*dim):
            nei_dict[s] = make_closure(n)
    return nei_dict

def get_neighbour_indices(i, j, k, neighbour_dict):
    return [f(i,j,k) for d,f in neighbour_dict.items()]

def add_neighbours(state, neighbour_dict):
    new_state = state.copy()
    for point in state.keys():
        for tup in get_neighbour_indices(*point, neighbour_dict):
            if not tup in state.keys():
                new_state[tup] = False
    return new_state

In [252]:
nebs3 = [n for n in itertools.product([1, 0, -1], repeat=3) if n!=(0,0,0)]
def get_neighbour_indices(i, j, k):
    pt = np.array([i, j, k])
    return [tuple(pt+n) for n in nebs3]

In [233]:
nebs4 = list(itertools.product([1, 0, -1], repeat=4))
def get_neighbour_indices_4(i, j, k, l):
    pt = np.array([i, j, k, l])
    return [tuple(pt+n) for n in nebs4 if n != (0,0,0,0)]

In [248]:
def print_layer(state, z):
    layer = {(k[0], k[1]):'#' if v else '.' for k,v in state.items() if k[2]==z}
    min_r = min([tup[0] for tup in layer.keys()])
    max_r = max([tup[0] for tup in layer.keys()])
    min_c = min([tup[1] for tup in layer.keys()])
    max_c = max([tup[1] for tup in layer.keys()])
    for r in range(min_r, max_r+1):
        print(''.join([layer[(r,c)] for c in range(min_c, max_c+1)]))
            

neighbour_dict = {'dnw': lambda i,j,k: (i-1, j-1,k-1), 'dn': lambda i,j,k: (i-1, j,k-1), 'dne': lambda i,j,k: (i-1, j+1,k-1),
              'dw': lambda i,j,k: (i, j-1,k-1), 'de': lambda i,j,k: (i, j+1,k-1), 'dsw': lambda i,j,k: (i+1, j-1,k-1), 
              'ds': lambda i,j,k: (i+1, j,k-1), 'dse': lambda i,j,k: (i+1, j+1,k-1), 'd': lambda i,j,k: (i, j,k-1),
              
             'nw': lambda i,j,k: (i-1, j-1,k), 'n': lambda i,j,k: (i-1, j,k), 'ne': lambda i,j,k: (i-1, j+1,k),
              'w': lambda i,j,k: (i, j-1,k), 'e': lambda i,j,k: (i, j+1,k), 'sw': lambda i,j,k: (i+1, j-1,k), 
              's': lambda i,j,k: (i+1, j,k), 'se': lambda i,j,k: (i+1, j+1,k),
             
              'unw': lambda i,j,k: (i-1, j-1,k+1), 'un': lambda i,j,k: (i-1, j,k+1), 'une': lambda i,j,k: (i-1, j+1,k+1),
              'uw': lambda i,j,k: (i, j-1,k+1), 'ue': lambda i,j,k: (i, j+1,k+1), 'usw': lambda i,j,k: (i+1, j-1,k+1), 
              'bs': lambda i,j,k: (i+1, j,k+1), 'use': lambda i,j,k: (i+1, j+1,k+1), 'u': lambda i,j,k: (i, j,k+1),
             }

def get_neighbour_indices(i, j, k):
    return [f(i,j,k) for d,f in neighbour_dict.items()]

def add_neighbours(state):
    new_state = state.copy()
    for point in state.keys():
        for tup in get_neighbour_indices(*point):
            if not tup in state.keys():
                new_state[tup] = False
    return new_state

In [231]:
state = get_input(test_data)
new_state = {}
for i in range(6):
    state = add_neighbours(state)
    new_state = state.copy()
    for point, is_active in state.items():
        n_active = 0
        for tup in get_neighbour_indices(*point):
            if tup in state.keys():
                n_active += state[tup]
            
        if is_active and n_active not in [2, 3]:
            new_state[point] = False
        elif ~is_active and n_active == 3:
            new_state[point] = True
    state = new_state.copy()

In [232]:
 print(sum([v for v in state.values()]))

112


In [310]:
t0 = timer()
state = get_input()
new_state = {}
t_add_n = 0
t_get_active = 0
n_add_n = 0
n_get_active = 0

neighbour_dict = prepare_dict(3)
for i in range(6):
    t1 = timer()
    state = add_neighbours(state, neighbour_dict)
    t_add_n += timer()- t1
    n_add_n += 1
    new_state = state.copy()
    for point, is_active in state.items():
        n_active = 0
        t1 = timer()
        for tup in get_neighbour_indices(*point, neighbour_dict):
            if tup in state.keys():
                n_active += state[tup]
        t_get_active += timer()-t1
        n_get_active += 1
        
        if is_active and n_active not in [2, 3]:
            new_state[point] = False
        elif ~is_active and n_active == 3:
            new_state[point] = True
    state = new_state.copy()

print(sum([v for v in state.values()]))
print(f'{round(timer()-t0, 1)} sec')

338
0.8 sec


## Part II

In [237]:
def add_neighbours_4(state):
    new_state = state.copy()
    for point in state.keys():
        for tup in get_neighbour_indices_4(*point):
            if not tup in state.keys():
                new_state[tup] = False
    return new_state

In [245]:
t0 = timer()
state = get_input_2()
new_state = {}
t_add_n = 0
t_get_active = 0
n_add_n = 0
n_get_active = 0
for i in range(6):
    t1 = timer()
    state = add_neighbours_4(state)
    t_add_n += timer()- t1
    n_add_n += 1
    new_state = state.copy()
    for point, is_active in state.items():
        n_active = 0
        t1 = timer()
        for tup in get_neighbour_indices_4(*point):
            if tup in state.keys():
                n_active += state[tup]
        t_get_active += timer()-t1
        n_get_active += 1
        
        if is_active and n_active not in [2, 3]:
            new_state[point] = False
        elif ~is_active and n_active == 3:
            new_state[point] = True
    state = new_state.copy()

print(sum([v for v in state.values()]))
print(f'{round(timer()-t0, 1)} sec')

2440
162.5 sec
add neighbours: 53.1 sec for 6 times
get active neighbours: 53.1 sec for 141644 times


In [246]:
print(f'add neighbours: {round(t_add_n, 1)} sec for {n_add_n} times')
print(f'get active neighbours: {round(t_get_active, 1)} sec for {n_get_active} times')

add neighbours: 53.1 sec for 6 times
get active neighbours: 109.1 sec for 141644 times


# Better solution!

In [34]:
import itertools
from timeit import default_timer as timer

def get_input(test_data=None, dim=3):
    data = test_data if test_data is not None else open('input.txt').read()
    state = {}
    lists = [list(n) for n in data.split('\n')]
    for i in range(len(lists)):
        for j in range(len(lists[0])):
            tup = (i, j, 0) if dim==3 else (i,j,0,0)
            state[tup] = (lists[i][j] == '#')
    return state

def make_closure(n):
    if len(n) == 3:
        return lambda i,j,k: (i+n[0], j+n[1], k+n[2])
    elif len(n) == 4:
        return lambda i,j,k,l: (i+n[0], j+n[1], k+n[2], l+n[3])
    else:
        raise NotImplementedError
        
def prepare_dict(dim):
    nei_dict = {}
    for s,n in enumerate(itertools.product([1, 0, -1], repeat=dim)):
        if n!=tuple([0]*dim):
            nei_dict[s] = make_closure(n)
    return nei_dict

def get_neighbour_indices(point, neighbour_dict):
    return [f(*point) for f in neighbour_dict.values()]

def add_neighbours(state, neighbour_dict):
    new_state = state.copy()
    for point in state.keys():
        for tup in get_neighbour_indices(point, neighbour_dict):
            if not tup in state.keys():
                new_state[tup] = False
    return new_state

def solve(dim):
    state = get_input(dim=dim)
    neighbour_dict = prepare_dict(dim)
    
    for i in range(6):
        state = add_neighbours(state, neighbour_dict)
        new_state = state.copy()
        for point, is_active in state.items():
            n_active = 0
            for tup in get_neighbour_indices(point, neighbour_dict):
                if tup in state.keys():
                    n_active += state[tup]

            if is_active and n_active not in [2, 3]:
                new_state[point] = False
            elif ~is_active and n_active == 3:
                new_state[point] = True
        state = new_state.copy()
    
    return sum([v for v in state.values()])
    

In [35]:
solve(dim=3)

338

## Part II

In [36]:
solve(dim=4)

2440